In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load Model

In [19]:
MODEL_ID = "GanjinZero/biobart-v2-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast = True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, 
    device_map = "auto", torch_dtype = "auto")


# Some Medical Text

In [20]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. Cessation of non-steroidal anti-inflammatory drug use along with steroid treatment resulted in a moderate
improvement in renal function, though residual impairment remained. Urine heavy metal screen returned with elevated levels of
urine copper, but with normal ceruloplasmin level. Workup suggested that the elevated copper levels were due to cirrhosis from
non-alcoholic fatty liver disease. The membranous glomerulonephritis is possibly linked to non-steroidal anti-inflammatory drug
exposure, and possibly to heavy metal exposure, and is clinically and pathologically much less likely to be a primary membranous
glomerulonephritis with negative serological markers.

Keywords: Minimal change disease, podocytopathy, secondary membranous glomerulonephritis, acute interstitial nephritis, non-steroidal anti-inflammatory drugs
"""

In [21]:
print(len(tokenizer.encode(medical_text)))

372


# Inference

In [22]:
inputs = tokenizer(medical_text, return_tensors = "pt", max_length = 1024, truncation = True, padding = True).to(model.device)                # move tensors to same device as the model

with torch.no_grad():
    generated = model.generate(**inputs,
        max_new_tokens = 64, num_beams = 4, length_penalty = 1.5, early_stopping = False, no_repeat_ngram_size = 3, encoder_no_repeat_ngram_size = 4)

summary = tokenizer.decode(generated[0], skip_special_tokens = True)
print(summary)


¯¯¯¯Non-steroidally anti-inflammatory Drugs are not only powerful analgesics and antipyroretics but still nephrotoxins and may cause                         aditionalelectrolytes disarray. IN addition tothe commonly expected adverse effects, including hyperuricemia, hyponatremic, acute renal ischemia, aditionalrenal
